In [2]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

In [3]:
movie_emoji_examples = [
    {"movie": "The Matrix", "emojis": "🕶️💊🤖"},
    {"movie": "The Lion King", "emojis": "🦁👑🌅"},
    {"movie": "Titanic", "emojis": "🚢❤️🧊"},
    {"movie": "Star Wars", "emojis": "⭐🔫🚀"},
    {"movie": "Jurassic Park", "emojis": "🦖🏝️🧬"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Give me three emojis that represent the movie: {movie}"),
        ("ai", "{emojis}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=movie_emoji_examples,
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie expert who represents movies using exactly three emojis. Always respond with exactly three emojis that best capture the theme, characters, or plot of the movie. No words or explanations.",
        ),
        few_shot_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "Give me three emojis that represent the movie: {movie}"),
    ]
)

In [4]:
def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(movie_title):
    result = chain.invoke({"movie": movie_title})

    memory.save_context(
        {"input": f"Give me three emojis that represent the movie: {movie_title}"},
        {"output": result.content},
    )

    return result.content


def ask_about_history(question):
    result = chain.invoke({"movie": question})

    memory.save_context({"input": question}, {"output": result.content})

    return result.content

In [ ]:
invoke_chain("Top Gun")

In [ ]:
invoke_chain("The Godfather")

In [ ]:
ask_about_history("What was the first movie I asked about?")